In [7]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import numpy as np
from PIL import Image
import os

In [8]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path)
            img = img.resize((224, 224))  # Resize images if needed
            img_array = np.array(img)
            images.append(img_array.flatten())  # Flatten the image array
            labels.append(label)
        except Exception as e:
            print(f"Error loading image: {img_path} - {e}")
    return np.array(images), np.array(labels)

In [9]:
autistic_folder = r"C:\Users\Shriram\Downloads\AutismDataset\train\Autistic"
non_autistic_folder = r"C:\Users\Shriram\Downloads\AutismDataset\train\Non_Autistic"

In [10]:
X_autistic, y_autistic = load_images_from_folder(autistic_folder, 1)

In [11]:
X_non_autistic, y_non_autistic = load_images_from_folder(non_autistic_folder, 0)

In [12]:
X = np.concatenate((X_autistic, X_non_autistic), axis=0)
y = np.concatenate((y_autistic, y_non_autistic), axis=0)

In [13]:
label_encoder = preprocessing.LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [15]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [16]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [ ]:
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

In [18]:
y_pred_proba = bst.predict(X_test, num_iteration=bst.best_iteration)

In [19]:
y_pred_class = [1 if x>=0.5 else 0 for x in y_pred_proba]

In [20]:
y_test_decoded = label_encoder.inverse_transform(y_test)

In [21]:
y_pred_decoded = label_encoder.inverse_transform(y_pred_class)

In [22]:
accuracy = accuracy_score(y_test_decoded,y_pred_decoded)

In [ ]:
print(accuracy)
